In [ ]:
# PYSPARK
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("Netflix_ETL").getOrCreate()

# Read csv file

%time netflix_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/tables/netflix_titles_2021-2.csv")

%time netflix_df.display(10)

In [ ]:
# Get the number of rows and columns in the dataframe
%time netflix_df.count(), len(netflix_df.columns)


In [ ]:
from pyspark.sql.functions import expr

# Define the new column names
new_columns = {'type':'tipo', 'title':'titulo', 'director':'diretor', 'cast':'elenco',
            'country':'pais', 'date_added':'adicionado_em','release_year':'ano_lancamento',
            'rating':'avaliacao', 'duration':'duracao', 'listed_in':'categorias', 'description':'descricao'}

# Use expr to rename the columns
%time netflix_df = netflix_df.select([expr(f"`{col}` as `{new_col}`") for col, new_col in new_columns.items()])

%time netflix_df.display(10)

In [ ]:
# NÂO CONSEGUI

from pyspark.sql.functions import isnan, when, count, round

# Count the number of missing values in each column
frequencia = netflix_df.select([count(when(isnan(c), c)).alias(c) for c in netflix_df.columns]).toPandas().transpose()
%time frequencia = frequencia.rename(columns={0:'Frequência'})

# Calculate the percentage of missing values in each column
total_rows = netflix_df.count()
percentual_na = round((frequencia / total_rows) * 100, 2)
%time percentual_na = percentual_na.rename(columns={'Frequência': '% de NAs'})

# Combine the count and percentage of missing values
valores_na = frequencia.join(percentual_na)

print(valores_na)

In [ ]:
from pyspark.sql.types import ShortType

%time netflix_df = netflix_df.withColumn("ano_lancamento", netflix_df["ano_lancamento"].cast(ShortType()))

In [ ]:
%time netflix_df.printSchema()